In [2]:
!pip install duckdb

/bin/bash: /home/developer/Downloads/data_engineering_practices/myenv/bin/pip: /home/developer/Downloads/data_engineering_practices_1/myenv/bin/python3: bad interpreter: No such file or directory


In [3]:
import os
import duckdb

BASE_DIR = os.getcwd()

csv_file = os.path.join(BASE_DIR, "data", "Electric_Vehicle_Population_Data.csv")
print("CSV path:", csv_file)

con = duckdb.connect()

# 1st, creating table
con.execute("""
CREATE TABLE IF NOT EXISTS electric_vehicles (
    VIN VARCHAR,
    County VARCHAR,
    City VARCHAR,
    State CHAR(2),
    "Postal Code" INTEGER,
    "Model Year" INTEGER,
    Make VARCHAR,
    Model VARCHAR,
    "Electric Vehicle Type" VARCHAR,
    "Clean Alternative Fuel Vehicle (CAFV) Eligibility" VARCHAR,
    "Electric Range" INTEGER,
    "Base MSRP" DOUBLE,
    "Legislative District" INTEGER,
    "DOL Vehicle ID" BIGINT,
    "Vehicle Location" VARCHAR,
    "Electric Utility" VARCHAR,
    "2020 Census Tract" BIGINT
)
""")

# 2nd, loading data into the table from csv file
con.execute(f"""
COPY electric_vehicles
FROM '{csv_file}'
(HEADER, DELIMITER ',')
""")

result = con.execute("SELECT * FROM electric_vehicles LIMIT 5").fetchdf()
# print(result)

schema = con.execute("DESCRIBE electric_vehicles").fetchdf()
print(schema)

CSV path: /home/developer/Downloads/data_engineering_practices/Exercises/Exercise-8/data/Electric_Vehicle_Population_Data.csv
                                          column_name column_type null   key  \
0                                                 VIN     VARCHAR  YES  None   
1                                              County     VARCHAR  YES  None   
2                                                City     VARCHAR  YES  None   
3                                               State     VARCHAR  YES  None   
4                                         Postal Code     INTEGER  YES  None   
5                                          Model Year     INTEGER  YES  None   
6                                                Make     VARCHAR  YES  None   
7                                               Model     VARCHAR  YES  None   
8                               Electric Vehicle Type     VARCHAR  YES  None   
9   Clean Alternative Fuel Vehicle (CAFV) Eligibility     VARCHAR  YES  No

In [4]:
# Question 3
# part 1
electric_cars_per_city = con.execute("""
SELECT City, COUNT("Electric Vehicle Type") AS electric_vehicle_count
FROM electric_vehicles
GROUP BY City
""").fetchdf()

print(electric_cars_per_city)


            City  electric_vehicle_count
0         Yakima                     297
1         Naches                      11
2      Anacortes                     267
3       Tumwater                     258
4         Burien                     392
..           ...                     ...
535      Matlock                       1
536     Nahcotta                       1
537  Westminster                       1
538      Lincoln                       1
539  Rock Island                       1

[540 rows x 2 columns]


In [ ]:
# part 2
top_3_vehicles = con.execute("""
    SELECT "Make", "Model", COUNT(*) AS total_count
    FROM electric_vehicles
    GROUP BY "Make", "Model"
    ORDER BY total_count DESC
    LIMIT 3
""").fetchdf()

print(top_3_vehicles)

     Make    Model  total_count
0   TESLA  MODEL 3        14511
1   TESLA  MODEL Y        12503
2  NISSAN     LEAF         7974


In [34]:
# part 3
most_p_with_postalcode = con.execute("""
SELECT "Make", "Model", "Postal Code", Total_count 
                                     FROM (
                                     SELECT "Make",
                                     "Model",
                                     "Postal Code",
                                     COUNT(*) as Total_count,
                                     ROW_NUMBER() OVER (Partition by "Postal Code" Order by count(*) DESC) as rn
                                     from electric_vehicles
                                     group by "Make", "Model", "Postal Code"                 
    ) sub
where rn=1
ORDER BY "POSTAL CODE"                             
""").fetch_df()
print(most_p_with_postalcode)

          Make        Model  Postal Code  Total_count
0    CHEVROLET         VOLT         1730            1
1        TESLA      MODEL 3         1731            1
2        TESLA      MODEL Y         1824            1
3       NISSAN         LEAF         2842            1
4        TESLA      MODEL 3         3804            1
..         ...          ...          ...          ...
641  CHEVROLET         VOLT        99361            2
642      TESLA      MODEL 3        99362           26
643     TOYOTA   RAV4 PRIME        99402            1
644     TOYOTA  PRIUS PRIME        99403            5
645       FORD       FUSION        99701            1

[646 rows x 4 columns]


In [ ]:
# part 4

BASE_DIR = os.getcwd()
output_folder = os.path.join(BASE_DIR, "reports", "electric_cars_by_year_parquet")

query = """
(SELECT "Make","Model", count("Make") as electric_car_count, "Model Year"
                                    from electric_vehicles
                                    group by "Model Year", "Make", "Model"
                                    order by "Model Year")"""
result_df = con.execute(query).fetchdf()
print(result_df)

for year, df_year in result_df.groupby("Model Year"):
    year_folder = os.path.join(output_folder, f"Model_Year={year}")
    os.makedirs(year_folder, exist_ok=True)
    df_year.to_parquet(os.path.join(year_folder, "data.parquet"), engine="pyarrow", index=False)


# con.execute(f"""
# COPY ({query}) 
# TO '{output_folder}' 
# (PARQUET, PARTITION_BY ('Model Year'))
# """)




                     Make            Model  electric_car_count  Model Year
0                    FORD           RANGER                   3        1999
1                    FORD           RANGER                   7        2000
2                   TESLA         ROADSTER                  13        2008
3                   TESLA         ROADSTER                  17        2010
4    WHEEGO ELECTRIC CARS           WHEEGO                   2        2010
..                    ...              ...                 ...         ...
386              CADILLAC            LYRIQ                   7        2023
387                  FORD          TRANSIT                  48        2023
388         MERCEDES-BENZ  EQS-CLASS SEDAN                  17        2023
389                JAGUAR           I-PACE                   1        2023
390               HYUNDAI           TUCSON                   1        2023

[391 rows x 4 columns]
